## Load Libraries

In [1]:
import os
import openai

In [2]:
import numpy as np
import pandas as pd

### Load API Key

In [18]:
openai.api_key = open('apikey.txt', 'r').read().strip('\n')

### Load and Explore Data

In [4]:
df_a = pd.read_csv('table_A.csv')
df_b = pd.read_csv('table_B.csv')
df_template = pd.read_csv('template.csv')

In [5]:
df_a.head()

,Date_of_Policy,FullName,Insurance_Plan,Policy_No,Monthly_Premium,Department,JobTitle,Policy_Start,Full_Name,Insurance_Type,Policy_Num,Monthly_Cost
0,05/01/2023,John Doe,Gold Plan,AB-12345,150.0,IT,Software Engineer,05/01/2023,John Doe,Gold,AB-12345,150.0
1,05/02/2023,Jane Smith,Silver Plan,CD-67890,100.0,HR,HR Manager,05/02/2023,Jane Smith,Silver,CD-67890,100.0
2,05/03/2023,Michael Brown,Bronze Plan,EF-10111,50.0,Marketing,Marketing Coordinator,05/03/2023,Michael Brown,Bronze,EF-10111,50.0
3,05/04/2023,Alice Johnson,Gold Plan,GH-12121,150.0,Finance,Financial Analyst,05/04/2023,Alice Johnson,Gold,GH-12121,150.0
4,05/05/2023,Bob Wilson,Silver Plan,IJ-13131,100.0,Sales,Sales Executive,05/05/2023,Bob Wilson,Silver,IJ-13131,100.0


In [6]:
df_a.describe()

,Monthly_Premium,Monthly_Cost
count,10.000000,10.000000
mean,105.000000,105.000000
std,43.779752,43.779752
min,50.000000,50.000000
25%,62.500000,62.500000
50%,100.000000,100.000000
75%,150.000000,150.000000
max,150.000000,150.000000


In [7]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date_of_Policy   10 non-null     object 
 1   FullName         10 non-null     object 
 2   Insurance_Plan   10 non-null     object 
 3   Policy_No        10 non-null     object 
 4   Monthly_Premium  10 non-null     float64
 5   Department       10 non-null     object 
 6   JobTitle         10 non-null     object 
 7   Policy_Start     10 non-null     object 
 8   Full_Name        10 non-null     object 
 9   Insurance_Type   10 non-null     object 
 10  Policy_Num       10 non-null     object 
 11  Monthly_Cost     10 non-null     float64
dtypes: float64(2), object(10)
memory usage: 1.1+ KB


In [12]:
Date_of_Policy = df_a['Date_of_Policy'].tolist()
Policy_Start = df_a['Policy_Start'].tolist()
Monthly_Premium = df_a['Monthly_Premium'].tolist()
Monthly_Cost = df_a['Monthly_Cost'].tolist()

print(f"Date of Policy is the same as Policy Start: {Date_of_Policy == Policy_Start}")
print(f"Monthly Premium is the same as Monthly Cost: {Monthly_Premium == Monthly_Cost}")

Date of Policy is the same as Policy Start: True
Monthly Premium is the same as Monthly Cost: True


In [8]:
df_b.head()

,PolicyDate,Name,PlanType,Policy_ID,PremiumAmount,Hobby,MaritalStatus,StartDate,Employee_Name,Plan_Name,PolicyID,Cost
0,2023-05-01,Doe,John,GoldPackage,AB12345,Photography,Married,2023-05-01,John Doe,Gold,AB12345,150
1,2023-05-02,Smith,Jane,SilverPackage,CD67890,Reading,Single,2023-05-02,Jane Smith,Silver,CD67890,100
2,2023-05-03,Brown,Michael,BronzePackage,EF10111,Traveling,Divorced,2023-05-03,Michael Brown,Bronze,EF10111,50
3,2023-05-04,Johnson,Alice,GoldPackage,GH12121,Cooking,Married,2023-05-04,Alice Johnson,Gold,GH12121,150
4,2023-05-05,Wilson,Bob,SilverPackage,IJ13131,Hiking,Single,2023-05-05,Bob Wilson,Silver,IJ13131,100


In [9]:
df_b.describe()

,Cost
count,10.000000
mean,105.000000
std,43.779752
min,50.000000
25%,62.500000
50%,100.000000
75%,150.000000
max,150.000000


In [10]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PolicyDate     10 non-null     object
 1   Name           10 non-null     object
 2   PlanType       10 non-null     object
 3   Policy_ID      10 non-null     object
 4   PremiumAmount  10 non-null     object
 5   Hobby          10 non-null     object
 6   MaritalStatus  10 non-null     object
 7   StartDate      10 non-null     object
 8   Employee_Name  10 non-null     object
 9   Plan_Name      10 non-null     object
 10  PolicyID       10 non-null     object
 11  Cost           10 non-null     int64 
dtypes: int64(1), object(11)
memory usage: 1.1+ KB


In [14]:
PolicyDate = df_b['PolicyDate'].tolist()
StartDate = df_b['StartDate'].tolist()
PremiumAmount = df_b['PremiumAmount'].tolist()
PolicyID = df_b['PolicyID'].tolist()

print(f"Policy Date and Start Date are the same: {PolicyDate == StartDate}")
print(f"Premium Amount and Policy ID are the same: {PremiumAmount == PolicyID}")

Policy Date and Start Date are the same: True
Premium Amount and Policy ID are the same: True


### Data Irregularities
1. Datatime objects were not converted correctly into datetime objects
2. cost in table B is an integer, whereas in table A it is a float (decimal)
3. Monthly Premium and Monthly Cost in table A are the same
4. Date of Policy and Policy Start in table A are the same

5. Policy Date and Start Date are the same in Table B
6. Name + Plan Type = Employee Name in Table B
7. Policy_ID and Plan_Name are the same with Policy_ID having 'Package' appended to it
8. Premium Amount and Policy ID are the same in Table B

## Strategy and Query ChatGPT

1. Ask ChatGPT to find similar columns between the two tables
2. Ask ChatGPT to transform the data, so it fits into a single table with the following columns<br>
Date	EmployeeName	Plan	PolicyNumber	Premium

In [15]:
def get_completion(message_history, model="gpt-3.5-turbo"):
    """
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=message_history,
        temperature=0, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]

In [20]:
message_history = []

# Let ChatGPT know what the task is
prompt = 'I have two tables, can you find which columns are similar to one another'
message_history.append({"role": "user", "content": prompt})
response = get_completion(message_history)
message_history.append({"role": "assistant", "content": response})

# Provide data
prompt = f'Table A ```{df_a}``` and Table B ```{df_b}```'
message_history.append({"role": "user", "content": prompt})
response = get_completion(message_history)
message_history.append({"role": "assistant", "content": response})

# Transform Data into a single table
prompt = 'Can you transform the data from both tables so it fits into a table with the following columns, and return it to me <table-columns>Date, EmployeeName, Plan, PolicyNumber, Premium </table-columns'
message_history.append({"role": "user", "content": prompt})
response = get_completion(message_history)
message_history.append({"role": "assistant", "content": response})

print(response)




Certainly! I will transform the data from both tables to fit into a table with the requested columns. Here is the resulting table:

```
|    Date    | EmployeeName |  Plan  | PolicyNumber | Premium |
|------------|--------------|--------|--------------|---------|
| 05/01/2023 |   John Doe   |  Gold  |   AB-12345   |  150.0  |
| 05/02/2023 |  Jane Smith  | Silver |   CD-67890   |  100.0  |
| 05/03/2023 | Michael Brown | Bronze |   EF-10111   |   50.0  |
| 05/04/2023 | Alice Johnson |  Gold  |   GH-12121   |  150.0  |
| 05/05/2023 |  Bob Wilson  | Silver |   IJ-13131   |  100.0  |
| 05/06/2023 | Carol Martinez| Bronze |   KL-14141   |   50.0  |
| 05/07/2023 | David Anderson|  Gold  |   MN-15151   |  150.0  |
| 05/08/2023 |  Eva Thomas  | Silver |   OP-16161   |  100.0  |
| 05/09/2023 | Frank Jackson | Bronze |   QR-17171   |   50.0  |
| 05/10/2023 |  Grace White  |  Gold  |   ST-18181   |  150.0  |
```

Please note that the table includes the columns Date, EmployeeName, Plan, PolicyNumbe

In [22]:
# Single Table Response
message_history = []

# Let ChatGPT know what the task is
prompt = 'I have a tables, can you find which columns are similar to one another'
message_history.append({"role": "user", "content": prompt})
response = get_completion(message_history)
message_history.append({"role": "assistant", "content": response})

# Provide data
prompt = f'Here is the table <table>{df_a}</table>'
message_history.append({"role": "user", "content": prompt})
response = get_completion(message_history)
message_history.append({"role": "assistant", "content": response})

# Transform Data into a single table
prompt = 'Can you transform the data from the table so it fits into a table with the following columns, and return it to me <table-columns>Date, EmployeeName, Plan, PolicyNumber, Premium </table-columns> using html format.'
message_history.append({"role": "user", "content": prompt})
response = get_completion(message_history)
message_history.append({"role": "assistant", "content": response})

print(response)

Certainly! I will transform the data from the provided table into a new table with the requested columns. Here is the transformed table:

<table>
  <tr>
    <th>Date</th>
    <th>EmployeeName</th>
    <th>Plan</th>
    <th>PolicyNumber</th>
    <th>Premium</th>
  </tr>
  <tr>
    <td>05/01/2023</td>
    <td>John Doe</td>
    <td>Gold Plan</td>
    <td>AB-12345</td>
    <td>150.0</td>
  </tr>
  <tr>
    <td>05/02/2023</td>
    <td>Jane Smith</td>
    <td>Silver Plan</td>
    <td>CD-67890</td>
    <td>100.0</td>
  </tr>
  <tr>
    <td>05/03/2023</td>
    <td>Michael Brown</td>
    <td>Bronze Plan</td>
    <td>EF-10111</td>
    <td>50.0</td>
  </tr>
  <tr>
    <td>05/04/2023</td>
    <td>Alice Johnson</td>
    <td>Gold Plan</td>
    <td>GH-12121</td>
    <td>150.0</td>
  </tr>
  <tr>
    <td>05/05/2023</td>
    <td>Bob Wilson</td>
    <td>Silver Plan</td>
    <td>IJ-13131</td>
    <td>100.0</td>
  </tr>
  <tr>
    <td>05/06/2023</td>
    <td>Carol Martinez</td>
    <td>Bronze Plan</td>
  